# Battery Module Structural Analysis

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/russlib/SegmentHandCalcs/blob/main/FSAESegmentHandCalcs.ipynb)

These calculations were used to calculate and verify the mechanical structure of our segment during the intial phases of design. 

This document was made to both include current SES calculations, SES calculations that were to be introduced in 2026, and calculations deemed to help verify solid mechanical design. 

## Table of Contents
1. [Constants and Geometry](#1.-Constants-and-Geometry)
2. [Load Cases & Bending Stress](#2.-Load-Cases-&-Bending-Stress---Polycarbonate-Lid)
3. [Fasteners: Tension, Shear, and Bearing](#3.-Fasteners:-Tension,-Shear,-and-Bearing)
4. [Bond Strength Analysis](#4.-Bond-Strength-Analysis)
5. [Euler-Johnson Buckling Analysis](#5.-Euler-Johnson-Buckling-Analysis)
6. [Passive Thermal Properties](#6.-Passive-Thermal-Properties)
7. [Summary Results Table](#7.-Summary-Results-Table)
8. [Sources Referenced](#Sources-Referenced)

### 1. Constants and Geometry
We establish the physical foundations and material properties here.

             ____________________________________________________________
            /                                                           /|
           /                     POLYCARBONATE LID                     / |
          /___________________________________________________________/  |
         |                                                           |   | <--EneSegHeight
         |                                                           |   |      (105.6mm)
         |___________________________________________________________|   |
         |                                                           |  /
         |                                                           | / <--EneSegmentWidth
         |___________________________________________________________|/        (81.2mm)
           <------------------- EneSegmentDepth -------------------> 
                                 (417.0mm)

         _________________________________________________________   
      | | [ o o ]   [ o o ]   [ o o ]   [ o o ]   [ o o ]   [ o o ] | 
      | | [ o o ]   [ o o ]   [ o o ]   [ o o ]   [ o o ]   [ o o ] | 
      | | [ o o ]   [ o o ]   [ o o ]   [ o o ]   [ o o ]   [ o o ] |  (81.2mm)WIDTH
      | | [ o o ]   [ o o ]   [ o o ]   [ o o ]   [ o o ]           | 
      | |_________________________________________________________| |
      |                                                             |
        <---------------------- DEPTH (417.0mm) ------------------->

In [16]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [17]:
import math
import pandas as pd

# Conversion Constants
PSI_TO_PA = 6894.76
MM_TO_M = 1/1000
INCH_TO_M = 25.4/1000
G = 9.80665

# 1.1 Module Dimensions
ene_mod_width = 20.3 * MM_TO_M
ene_seg_height = 105.6 * MM_TO_M
ene_mod_depth = 69.5 * MM_TO_M
module_weight = 0.278  # kg

# 1.2 Segment Assembly
ene_segment_depth = 6 * ene_mod_depth #This represents the longest dimension of the segment
ene_segment_width = ene_mod_width * 4 #This repersents the width fromm 4 enepaq bricks side by side
#1x Enepaq bricks in height | 1 * ene_seg_height = Redundant
garolite_thickness = (1/8) * INCH_TO_M
segment_modules_weight = module_weight * 23 #total enepaq mass

# 1.3 Material Properties
garolite_strength = 38000 * PSI_TO_PA
garolite_modulus = 2400000 * PSI_TO_PA
poly_thickness = 3/8 * INCH_TO_M
poly_strength = 93e6  # Pa #Online Flexural Strength
poly_modulus = 2.21e9 # Pa

# 1.4 Area Calculations
# Areas Garolite - Imagine Section Views
top_cross_sec_area = ene_segment_depth * garolite_thickness * 2 #airflow "endplates" arent considerd valid area   # View Axis: Depth X, Width Y
width_cross_sec_area = ene_segment_depth * garolite_thickness * 1 #airflow "endplates" arent considerd valid area # View Axis: Depth X, Height Y
depth_cross_sec_area = (ene_segment_width * garolite_thickness * 2) + (ene_seg_height * garolite_thickness * 2)   # View Axis: Width X, Height Y


## 2. Load Cases & Bending Stress - Polycarbonate Lid
Calculated for 20g vertical and 40g lateral loads.

Assuming a uniformly distributed load where total force $F$ is related to the distributed load $w$ by $F = w \cdot L$:

**Maximum Bending Moment:**
$$ M_{max} = \frac{w L^2}{8} = \frac{F \cdot L}{8} $$

**Bending Stress:**
$$ \sigma = \frac{M \cdot c}{I} $$

**Maximum Deflection ($\delta_{max}$):**
$$ \delta_{max} = \frac{5 w L^4}{384 E I} = \frac{5 F \cdot L^3}{384 E I} $$

In [18]:
# Value Add: Verifies that our polycarbonate lid will be able to survive a 20g vertical crash load.
# The cells will not break the lid or fly out of their segment casing. 

# 2.1 Vertical Force (20g)
top_force = segment_modules_weight * 20 * G
# 2.2 Lateral/Longitudinal Forces (40g)
side_force = segment_modules_weight * 40 * G
long_force = segment_modules_weight * 40 * G
# 2.2 Bending Moment (Simply Supported Case)
m_a = (top_force * ene_segment_depth) / 8

# 2.3 Moment of Inertia for Lid
i_lid = (ene_segment_width * (poly_thickness**3)) / 12

# 2.4 Stress and SF
stress_bending_lid = (m_a * (poly_thickness / 2)) / i_lid
sf_lid = poly_strength / stress_bending_lid

# 2.5 Deflection Check
delta_max = (5 * top_force * ene_segment_depth**3) / (384 * poly_modulus * i_lid)

print(f"Lid Safety Factor: {sf_lid:.4f}")
#print(f"Max Lid Deflection: {delta_max * 1000:.2f} mm")

Lid Safety Factor: 1.7468


## 3. Fasteners


### Tensile Stress Area

If a threaded rod is subjected to pure tensile loading, its strength is defined by the average of the minor and pitch diameters. The **tensile-stress area**, $A_t$, is defined as:

$$ A_t = \frac{\pi}{4} \left( \frac{d_p + d_r}{2} \right)^2 \tag{15.1a} $$

**For ISO metric threads:**
$$ d_p = d - 0.649519p, \quad d_r = d - 1.226869p \tag{15.1c} $$

**Where:**
- $d$ = major/outside diameter (mm)
- $p$ = thread pitch (mm)

**Axial Tensile Stress:**
$$ \sigma_t = \frac{F}{A_t} \tag{15.2} $$

> *Ref: Norton, R. L. (2020). Machine Design: An Integrated Approach. 6th ed., p. 907.*

In [19]:
# 3.1 Bolt Geometry (M6) #calculates bolt maximums
dia = 6.0 * MM_TO_M
pitch = 1.0 * MM_TO_M
dia_pitch = dia - 0.649519 * pitch
dia_root = dia - 1.226869 * pitch
area_bolt_tensile = (math.pi / 4) * ((dia_pitch + dia_root) / 2)**2
max_tensile_fastener = 827370875 * area_bolt_tensile # Pa

# 3.2 SF Calculations (Assuming 3 bolts per end)
# Checks 20g Vertical Crash Load
sf_bolt_tension = (max_tensile_fastener * 3) / top_force

# 3.3 Plug Shear 
# Checks to see if G10 is the limiting factor in a 20g vertical crash load 
PlugShearArea = dia * garolite_thickness # assumes the 
Plug_stress = (top_force / 6) / PlugShearArea # Distributed across 6 bolt locations per casing
sf_plug = garolite_strength / Plug_stress

print(f"Bolt Tension SF: {sf_bolt_tension:.4f}")
print(f"Bolt Plug SF: {sf_plug:.4f}")

# 3.4 Bolt Shear SF (Assumes 3 bolts per side)
# Checks 40g horiztonal(l/r) crash loading against shear
width_force = 6 * segment_modules_weight * 40 * G #40g Horizontal Crash Load
sf_bolt_shear = (max_tensile_fastener * 3) / (width_force)

# 3.5 Tear-out Strength of G10
bolt_dist_edge = 8 / 1000
max_tearout_force = bolt_dist_edge * garolite_thickness * (38000 * PSI_TO_PA)
sf_tearout = max_tearout_force / (width_force / 6)

print(f"Bolt Shear SF: {sf_bolt_shear:.4f}")
print(f"Bolt Tear-out SF: {sf_tearout:.4f}")

Bolt Tension SF: 39.8290
Bolt Plug SF: 23.8795
Bolt Shear SF: 3.3191
Bolt Tear-out SF: 2.6533


## 4. Bond Strength Analysis
Given simple loading, how strong is the bonded G10?
This does not account for finger joints

In [20]:
# FR4 / G10 Bond Strength
# Assuming no mechanical latching (which we do have) how strongly is the G10 bonded together

# 4.1 Bond Strength
fr4_bond_strength = 2200 * PSI_TO_PA
depth_force = segment_modules_weight * 40 * G # horizontal(f/b) load case, 

# 4.2 Bond Areas
depth_restricting_bond_area = garolite_thickness * ene_seg_height
side_bond_area = 0.02 * garolite_thickness * 2 #2cm of sidewall-flange overlap
bond_max_force = fr4_bond_strength * (depth_restricting_bond_area + side_bond_area)
sf_bond = bond_max_force / depth_force

print(f"(Simple) Bond Safety Factor: {sf_bond:.4f}")

(Simple) Bond Safety Factor: 2.7957


## 5. Euler-Johnson Buckling Analysis

### Euler-Johnson Buckling Criteria ($SR = L/k$)

**Critical Slenderness and Radius of Gyration:**
$$ C_c = \pi \sqrt{\frac{2E}{\sigma_y}}, \quad k = \sqrt{\frac{I_{xx}}{A}} $$

**Johnson (Inelastic) Buckling if $SR < C_c$:**
$$ P_{cr} = A \left[ \sigma_y - \frac{1}{E} \left( \frac{\sigma_y SR}{2\pi} \right)^2 \right] $$

**Euler (Elastic) Buckling if $SR \ge C_c$:**
$$ P_{cr} = \frac{\pi^2 E I_{xx}}{L^2} $$

**Safety Factor:**
$$ SF = \frac{P_{cr}}{F_{applied}} $$

> *(Ref: Norton, Machine Design, 6th Ed., p. 231)*

In [21]:
# 5.1 Global Moment of Inertia (Ixx) Assembly
moment_inertia_walls = 2 * (((ene_segment_width/2 + garolite_thickness/2)**2 * garolite_thickness * ene_seg_height)) + (garolite_thickness**3 * ene_seg_height / 12)
moment_inertia_bottom = (garolite_thickness/2 + ene_seg_height/2)**2 * garolite_thickness * ene_segment_width + (garolite_thickness**3 * ene_segment_width / 12)
moment_inertia_lid = (garolite_thickness/2 + ene_seg_height/2 + 0.015)**2 * garolite_thickness * ene_segment_width + (garolite_thickness**3 * ene_segment_width / 12)
ixx = moment_inertia_walls + moment_inertia_bottom + moment_inertia_lid

# 5.2 Slenderness and Buckling Limits
radius_of_gyration = math.sqrt(ixx / depth_cross_sec_area)
slenderness_ratio = ene_segment_depth / radius_of_gyration
critical_slenderness = math.pi * math.sqrt((2 * garolite_modulus) / garolite_strength)

if slenderness_ratio < critical_slenderness:
    # Johnson Inelastic Buckling
    max_force_buckle = depth_cross_sec_area * (garolite_strength - (garolite_strength**2 * slenderness_ratio**2) / (4 * math.pi**2 * garolite_modulus))
    mode = "Johnson"
else:
    # Euler Elastic Buckling
    max_force_buckle = (math.pi**2 * garolite_modulus * ixx) / (ene_segment_depth**2)
    mode = "Euler"

sf_buckling = max_force_buckle / side_force
print(f"Buckling Mode: {mode}, SF: {sf_buckling:.4f}")

Buckling Mode: Johnson, SF: 120.7030


## 6. Passive Thermal Properties
Calculating the heat dissipation capacity of the segment casing.

In [22]:
# 6.1 Surface Area and Heat Transfer
surface_area = (ene_segment_depth * ene_segment_width * 2) + (ene_seg_height * ene_segment_depth * 2)
garolite_conductivity = 0.288 # W/m*K
watts_per_kelvin = (1 / garolite_thickness) * garolite_conductivity * surface_area

print(f"Total Segment Surface Area: {surface_area:.4f} m^2")
print(f"Passive Heat Transfer Rate: {watts_per_kelvin:.4f} W/K")

Total Segment Surface Area: 0.1558 m^2
Passive Heat Transfer Rate: 14.1316 W/K


## 7. Summary Results Table

In [23]:
summary_data = {
    "Calculation Ref": [
        "Section 2.4", "Section 3.2", "Section 3.3", 
        "Section 3.4", "Section 3.5", "Section 4.1", "Section 5.2"
    ],
    "Load Case": [
        "Lid - 20g Vertical", "Bolt Tension- 20g Vertical", "Bolt Plug-out - 20g Vertical", 
        "Bolt Shear - 40g Horizontal", "Bolt Tear-out - 40g Horizontal", "Bond Strength - 40g Horizontal", "Casing Buckling - 40g Horizontal"
    ],
    "Safety Factor": [
        sf_lid, sf_bolt_tension, sf_plug, 
        sf_bolt_shear, sf_tearout, sf_bond, sf_buckling
    ]
}

df = pd.DataFrame(summary_data).set_index("Calculation Ref")
display(df.round(4))

,Load Case,Safety Factor
Calculation Ref,,
Section 2.4,Lid - 20g Vertical,1.7468
Section 3.2,Bolt Tension- 20g Vertical,39.8290
Section 3.3,Bolt Plug-out - 20g Vertical,23.8795
Section 3.4,Bolt Shear - 40g Horizontal,3.3191
Section 3.5,Bolt Tear-out - 40g Horizontal,2.6533
Section 4.1,Bond Strength - 40g Horizontal,2.7957
Section 5.2,Casing Buckling - 40g Horizontal,120.7030


---
## Sources Referenced
1. [Enepaq VTC6 Module Datasheet](https://enepaq.com/wp-content/uploads/2023/09/VTC6-Sony-Murata-Li-ion-Battery-Module-With-Temperature-Sensor-Datasheet-Tesla-Battery-Sponsorship-Formula-SAE-Electric-Formula-Student-Battery-Pack.pdf) - Module dimensions and weight specifications.
2. [Laminated Plastics G-10/FR4 Data](https://laminatedplastics.com/g-10.pdf) - Material strengths, bonding data, and tear-out limits.
3. [MatWeb Polycarbonate Data](https://www.matweb.com/search/DataSheet.aspx?MatGUID=501acbb63cbc4f748faa7490884cdbca) - Mechanical properties for the module lid.